In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

print('pandas version is {}.'.format(pd.__version__))
print('numpy version is {}.'.format(np.__version__))
print('scikit-learn version is {}.'.format(sklearn.__version__))
print('seaborn version is {}.'.format(sns.__version__))
print('matplotlib version is {}.'.format(matplotlib.__version__))

In [ ]:
training_data = pd.read_csv("EH-training-data.csv", names = range(0, 59))
training_labels = pd.read_csv("EH-training-labels.csv", names = range(0, 3))
testing_data = pd.read_csv("EH-testing-data.csv", names = range(0, 59))
testing_labels = pd.read_csv("EH-testing-labels.csv", names = range(0, 3))
print "This set has {} training_data points and {} features.".format(*training_data.shape)
print "This set has {} training_label points and {} features.".format(*training_labels.shape)
print "This set has {} testing_data points and {} features.".format(*testing_data.shape)
print "This set has {} testing_label points and {} features.".format(*testing_labels.shape)
#display(training_data)
#display(training_labels)
#display(testing_data)
#display(testing_labels)

In [ ]:
training_data.describe()

In [ ]:
sns.set(style="white")
corr = training_data.corr()
mask = np.zeros_like(corr, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize = (22, 18))
cmap = sns.diverging_palette(255, 140, as_cmap = True)
sns.heatmap(corr, mask = mask, cmap = cmap, vmax = .3, 
            square = True, xticklabels = True, yticklabels = True, linewidths = 1, cbar_kws = {"shrink": .5}, ax = ax)

In [ ]:
#def features_hist(data):
#	"""This method ouputs a histograms of features."""
#	for i in data.columns:
#		stuff = pd.DataFrame(data = data[i], columns = [i])
#		stuff.plot(kind = 'hist', bins = 10)

In [ ]:
#features_hist(training_data)